In [ ]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)


def modify_paths(config):
    # dataset.data_dir に ../ を追加
    config['dataset']['data_dir'] = os.path.join('..', config['dataset']['data_dir'])

    # dataset.split_config が存在する場合に ../ を追加
    if 'split_config' in config['dataset']:
        config['dataset']['split_config'] = os.path.join('..', config['dataset']['split_config'])

    return config

In [ ]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/dataset/gen1/event_frame/sequence/dataset.yaml',
        '../config/model/rvt_detector/rvt_frame.yaml',
        '../config/experiment/sequence/train.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
config = modify_paths(merged_conf)
dynamically_modify_train_config(config=config)

In [ ]:
from modules.fetch import fetch_data_module, fetch_model_module

data = fetch_data_module(config)
data.setup('fit')
model = fetch_model_module(config)
model.setup('fit')

In [ ]:
data_loader = data.val_dataloader()
data_iter = iter(data_loader)

In [ ]:
sample = next(data_iter)

In [ ]:
rnn_model = model.model

In [ ]:
events = sample['events'][:, 0].float()
events.shape

In [ ]:
rnn_model.eval()

In [ ]:
outputs, states = rnn_model.forward_backbone(events, None, None)